In [2]:
import acquire, wrangle_zillow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.preprocessing
from sklearn.model_selection import train_test_split

df = wrangle_zillow.new_zillow_data()
df.shape

(77380, 70)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 70 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77380 non-null  int64  
 1   typeconstructiontypeid        222 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77380 non-null  float64
 4   heatingorsystemtypeid         49439 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      206 non-null    float64
 7   airconditioningtypeid         24953 non-null  float64
 8   id                            77380 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77380 non-null  float64
 11  bedroomcnt                    77380 non-null  float64
 12  buildingqualitytypeid         49671 non-null  float64
 13  c

In [4]:
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,logerror,pid,tdate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,...,0.025595,14297519,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,...,0.055619,17052889,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,...,0.005383,14186244,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,...,-0.103410,12177905,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,...,0.006940,10887214,2017-01-01,Central,None,None,Central,Condominium,None,None


# Lets get rid of a few columns

In [7]:
df = df.drop(columns=['parcelid','typeconstructiontypeid','storytypeid','propertylandusetypeid',
                      'heatingorsystemtypeid','buildingclasstypeid','architecturalstyletypeid',
                      'airconditioningtypeid','id','basementsqft','buildingqualitytypeid','decktypeid',
                      'finishedfloor1squarefeet','finishedsquarefeet12','finishedsquarefeet13',
                      'finishedsquarefeet50','finishedsquarefeet15','finishedsquarefeet6','fireplacecnt',
                      'garagetotalsqft','poolsizesum','pooltypeid10','pooltypeid2','pooltypeid7',
                      'propertycountylandusecode','propertyzoningdesc','rawcensustractandblock',
                      'regionidcounty','regionidneighborhood','threequarterbathnbr','unitcnt',
                      'yardbuildingsqft17','yardbuildingsqft26','fireplaceflag','structuretaxvaluedollarcnt',
                      'assessmentyear','landtaxvaluedollarcnt','taxdelinquencyflag','taxdelinquencyyear',
                      'censustractandblock','id','pid','airconditioningdesc','architecturalstyledesc',
                      'buildingclassdesc','heatingorsystemdesc','storydesc'])
df

,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,fips,fullbathcnt,garagecarcnt,hashottuborspa,latitude,longitude,...,regionidzip,roomcnt,yearbuilt,numberofstories,taxvaluedollarcnt,taxamount,logerror,tdate,propertylandusedesc,typeconstructiondesc
0,3.5,4.0,3.5,3100.0,6059.0,3.0,2.0,NaN,33634931.0,-117869207.0,...,96978.0,0.0,1998.0,NaN,1023282.0,11013.72,0.025595,2017-01-01,Single Family Residential,None
1,1.0,2.0,1.0,1465.0,6111.0,1.0,1.0,NaN,34449266.0,-119281531.0,...,97099.0,5.0,1967.0,1.0,464000.0,5672.48,0.055619,2017-01-01,Single Family Residential,None
2,2.0,3.0,2.0,1243.0,6059.0,2.0,2.0,NaN,33886168.0,-117823170.0,...,97078.0,6.0,1962.0,1.0,564778.0,6488.30,0.005383,2017-01-01,Single Family Residential,None
3,3.0,4.0,3.0,2376.0,6037.0,3.0,NaN,NaN,34245180.0,-118240722.0,...,96330.0,0.0,1970.0,NaN,145143.0,1777.51,-0.103410,2017-01-01,Single Family Residential,None
4,3.0,3.0,3.0,1312.0,6037.0,3.0,NaN,NaN,34185120.0,-118414640.0,...,96451.0,0.0,1964.0,NaN,119407.0,1533.89,0.006940,2017-01-01,Condominium,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77375,3.0,3.0,3.0,1741.0,6037.0,3.0,NaN,NaN,34202400.0,-118502000.0,...,96415.0,0.0,1980.0,NaN,379000.0,4685.34,-0.002245,2017-09-20,Condominium,None
77376,2.0,2.0,2.0,1286.0,6037.0,2.0,NaN,NaN,34245368.0,-118282383.0,...,96284.0,0.0,1940.0,NaN,354621.0,4478.43,0.020615,2017-09-20,Single Family Residential,None
77377,2.0,4.0,2.0,1612.0,6111.0,2.0,2.0,NaN,34300140.0,-118706327.0,...,97116.0,7.0,1964.0,1.0,67205.0,1107.48,0.013209,2017-09-21,Single Family Residential,None
77378,1.0,3.0,1.0,1032.0,6037.0,1.0,NaN,NaN,34040895.0,-118038169.0,...,96480.0,0.0,1954.0,NaN,49546.0,876.43,0.037129,2017-09-21,Single Family Residential,None


In [15]:
def get_counties():
    # create dummy vars of fips id
    county_df = pd.get_dummies(df.fips)
    # rename columns by actual county name
    county_df.columns = ['LA', 'Orange', 'Ventura']
    # concatenate the dataframe with the 3 county columns to the original dataframe
    df_dummies = pd.concat([df, county_df], axis = 1)
    # drop regionidcounty and fips columns
    df_dummies = df_dummies.drop(columns = ['fips'])
    return df_dummies

In [16]:
get_counties()

,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,fullbathcnt,garagecarcnt,hashottuborspa,latitude,longitude,lotsizesquarefeet,...,numberofstories,taxvaluedollarcnt,taxamount,logerror,tdate,propertylandusedesc,typeconstructiondesc,LA,Orange,Ventura
0,3.5,4.0,3.5,3100.0,3.0,2.0,NaN,33634931.0,-117869207.0,4506.0,...,NaN,1023282.0,11013.72,0.025595,2017-01-01,Single Family Residential,None,0,1,0
1,1.0,2.0,1.0,1465.0,1.0,1.0,NaN,34449266.0,-119281531.0,12647.0,...,1.0,464000.0,5672.48,0.055619,2017-01-01,Single Family Residential,None,0,0,1
2,2.0,3.0,2.0,1243.0,2.0,2.0,NaN,33886168.0,-117823170.0,8432.0,...,1.0,564778.0,6488.30,0.005383,2017-01-01,Single Family Residential,None,0,1,0
3,3.0,4.0,3.0,2376.0,3.0,NaN,NaN,34245180.0,-118240722.0,13038.0,...,NaN,145143.0,1777.51,-0.103410,2017-01-01,Single Family Residential,None,1,0,0
4,3.0,3.0,3.0,1312.0,3.0,NaN,NaN,34185120.0,-118414640.0,278581.0,...,NaN,119407.0,1533.89,0.006940,2017-01-01,Condominium,None,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77375,3.0,3.0,3.0,1741.0,3.0,NaN,NaN,34202400.0,-118502000.0,59487.0,...,NaN,379000.0,4685.34,-0.002245,2017-09-20,Condominium,None,1,0,0
77376,2.0,2.0,2.0,1286.0,2.0,NaN,NaN,34245368.0,-118282383.0,47405.0,...,NaN,354621.0,4478.43,0.020615,2017-09-20,Single Family Residential,None,1,0,0
77377,2.0,4.0,2.0,1612.0,2.0,2.0,NaN,34300140.0,-118706327.0,12105.0,...,1.0,67205.0,1107.48,0.013209,2017-09-21,Single Family Residential,None,0,0,1
77378,1.0,3.0,1.0,1032.0,1.0,NaN,NaN,34040895.0,-118038169.0,5074.0,...,NaN,49546.0,876.43,0.037129,2017-09-21,Single Family Residential,None,1,0,0


In [17]:
#what type of properties are in the df

df.propertylandusedesc.unique()

array(['Single Family Residential', 'Condominium',
       'Duplex (2 Units, Any Combination)', 'Residential General',
       'Planned Unit Development', 'Triplex (3 Units, Any Combination)',
       'Quadruplex (4 Units, Any Combination)',
       'Commercial/Office/Residential Mixed Used', 'Cluster Home',
       'Cooperative', 'Manufactured, Modular, Prefabricated Homes',
       'Mobile Home', 'Townhouse'], dtype=object)